# Survival kit Torch Distribution

In [13]:
import torch
import torch.distributions as dist
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions import Normal, Independent

Article

https://bochang.me/blog/posts/pytorch-distributions/

Doc Torch

https://docs.pytorch.org/docs/1.3.0/distributions.html#multivariatenormal

**Paramètres de la distribution:**

event_shape : dimension de la variable

batch_shape : batch size, indépendent, non identiques

**Arguments de rsample:**

sample_shape : nombre d'échantillons iid

In [ ]:
# Crée une distribution MVN
# - covariance_matrix est de dimension 2x2, donc la dimension de la variable aléatoire est 2 : event_shape = 2
# - loc est le vecteur de moyenne de dimension 2

mvn = MultivariateNormal(
    loc=torch.zeros(2),
    covariance_matrix=torch.tensor([[1.0, 0.5], [0.5, 1.0]])
)

samples = mvn.rsample((1000,))  # Génère 1000 échantillons

print(samples.shape)  # Affiche la forme des échantillons

torch.Size([1000, 2])


In [5]:
# crée une autre distribution MVN
# - covariance_matrix toujours 2x2 => event_shape = 2
# - loc déduit du tensor en input : shape (5, 3, 2) + event_shape = 2 => batch_shape = (5, 3)

mvn = MultivariateNormal(
    loc=torch.randn(5,3,2),
    covariance_matrix=torch.eye(2)
)

samples = mvn.rsample((10,))  # Génère 10 échantillons

print(samples.shape)  # Affiche la forme des échantillons

torch.Size([10, 5, 3, 2])


In [8]:
# crée une autre distribution MVN
# - covariance_matrix toujours 2x2 => event_shape = 2
# - loc déduit du tensor en input : shape (5, 3, 2) + event_shape = 2 => batch_shape = (5, 3)

try:
    mvn = MultivariateNormal(
        loc=torch.randn(5,3,4),
        covariance_matrix=torch.eye(2)
    )

    samples = mvn.rsample((10,))  # Génère 10 échantillons
except RuntimeError as e:
    print(f"Non correspondance entre la taille de la matrice de covariance et la dernière dimension de loc")

Non correspondance entre la taille de la matrice de covariance et la dernière dimension de loc


### Independent class

The Independent class does not represent any probability distribution. Instead, it creates a new distribution instance by “reinterpreting” some of the batch shapes of an existing distribution as event shapes.

torch.distributions.independent.Independent(base_distribution,
reinterpreted_batch_ndims, validate_args=None)

In [15]:
normal = Normal(
    loc = torch.randn(5,3,2),
    scale = torch.ones(5,3,2)
)

print(f"AVANT Independent:")
print(f"Caractéristiques de la distribution normale : batch_shape = {normal.batch_shape}, event_shape = {normal.event_shape}")

# use Independent pour recaster la dernière dimension de 'batch_shape' dans 'event_shape' 
independent_normal_1 = Independent(normal, 1) 
print(f"APRES Independent(normal, 1):")
print(f"Caractéristiques de la distribution normale indépendante : batch_shape = {independent_normal_1.batch_shape}, event_shape = {independent_normal_1.event_shape}")

# use Independent pour recaster les deux dernières dimensions de 'batch_shape' dans 'event_shape' 
independent_normal_2 = Independent(normal, 2) 
print(f"APRES Independent(normal, 2):")
print(f"Caractéristiques de la distribution normale indépendante : batch_shape = {independent_normal_2.batch_shape}, event_shape = {independent_normal_2.event_shape}")

AVANT Independent:
Caractéristiques de la distribution normale : batch_shape = torch.Size([5, 3, 2]), event_shape = torch.Size([])
APRES Independent(normal, 1):
Caractéristiques de la distribution normale indépendante : batch_shape = torch.Size([5, 3]), event_shape = torch.Size([2])
APRES Independent(normal, 2):
Caractéristiques de la distribution normale indépendante : batch_shape = torch.Size([5]), event_shape = torch.Size([3, 2])
